In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split


#make sure to be in microsoft store python

rotten_tomatoes_reviews = pd.read_csv('rotten_tomatoes_critic_reviews.csv')
semanlysis_dataset = pd.read_csv("IMDB Dataset.csv")

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pickle
# Assuming 'semanlysis_dataset' is your dataframe

# Split the data
train, test = train_test_split(semanlysis_dataset, test_size=0.25, train_size=.75, random_state=42)

# Define features and labels
train_x, train_y = train['review'], train['sentiment']
test_x, test_y = test['review'], test['sentiment']

# TF-IDF vectorization
tfidf = TfidfVectorizer(stop_words='english')
train_x_vector = tfidf.fit_transform(train_x)
test_x_vector = tfidf.transform(test_x)

with open ('model_tfidf', 'wb') as files:
    pickle.dump(tfidf, files)
# Define and train the model
log_reg = LogisticRegression()
log_reg.fit(train_x_vector, train_y)

# Evaluate the model
accuracy = log_reg.score(test_x_vector, test_y)
print(f"Accuracy: {accuracy}")

# Make predictions
prediction = log_reg.predict(tfidf.transform(['I did not like this movie at all']))
print(prediction)


with open ('model_log_reg', 'wb') as files:
    pickle.dump(log_reg, files)


Accuracy: 0.89576
['negative']


In [18]:
print(semanlysis_dataset.head())

                                               review sentiment
Id                                                             
1   One of the other reviewers has mentioned that ...  positive
2   A wonderful little production. <br /><br />The...  positive
3   I thought this was a wonderful way to spend ti...  positive
4   Basically there's a family where a little boy ...  negative
5   Petter Mattei's "Love in the Time of Money" is...  positive


In [6]:
columns_to_drop = ['rotten_tomatoes_link', 'critic_name','top_critic', 'publisher_name', 'review_type', 'review_date' ]
rotten_tomatoes_reviews = rotten_tomatoes_reviews.drop(columns=columns_to_drop)
rotten_tomatoes_reviews = rotten_tomatoes_reviews[~rotten_tomatoes_reviews['review_score'].isnull()]
rotten_tomatoes_reviews = rotten_tomatoes_reviews.dropna()

#Convert letters to values 
#and make defined scale and converter

print(rotten_tomatoes_reviews.head())

  review_score                                     review_content numerator  \
3        3.5/5  Whether audiences will get behind The Lightnin...       3.5   
6          1/4  Harry Potter knockoffs don't come more transpa...         1   
7        3.5/5  Percy Jackson isn't a great movie, but it's a ...       3.5   
8            B                         Fun, brisk and imaginative         B   
9          3/5  Crammed with dragons, set-destroying fights an...         3   

   denominator  bool_denominator  _4_column  _5_column  _10_column  
3            5             False      False       True       False  
6            4             False       True      False       False  
7            5             False      False       True       False  
8            0              True      False      False       False  
9            5             False      False       True       False  


In [7]:

rotten_tomatoes_reviews[['numerator', 'denominator']] = rotten_tomatoes_reviews['review_score'].str.split('/', expand=True)

print(rotten_tomatoes_reviews.head())

  review_score                                     review_content numerator  \
3        3.5/5  Whether audiences will get behind The Lightnin...       3.5   
6          1/4  Harry Potter knockoffs don't come more transpa...         1   
7        3.5/5  Percy Jackson isn't a great movie, but it's a ...       3.5   
8            B                         Fun, brisk and imaginative         B   
9          3/5  Crammed with dragons, set-destroying fights an...         3   

  denominator  bool_denominator  _4_column  _5_column  _10_column  
3           5             False      False       True       False  
6           4             False       True      False       False  
7           5             False      False       True       False  
8        None              True      False      False       False  
9           5             False      False       True       False  


In [8]:
import numpy as np
null_column = rotten_tomatoes_reviews['denominator'].isnull()
rotten_tomatoes_reviews['bool_denominator'] = null_column
rotten_tomatoes_reviews['denominator'] = rotten_tomatoes_reviews['denominator'].fillna(0)
rotten_tomatoes_reviews['denominator'] = pd.to_numeric(rotten_tomatoes_reviews['denominator'], errors='coerce')

rotten_tomatoes_reviews['denominator'] = rotten_tomatoes_reviews['denominator'].astype(int)
rotten_tomatoes_reviews['_4_column'] = np.where(rotten_tomatoes_reviews['denominator'] == 4, True, False)
rotten_tomatoes_reviews['_5_column'] = np.where(rotten_tomatoes_reviews['denominator'] == 5, True, False)
rotten_tomatoes_reviews['_10_column'] = np.where(rotten_tomatoes_reviews['denominator'] == 10, True, False)
rotten_tomatoes_reviews['_4_column'].fillna(False, inplace=True)
rotten_tomatoes_reviews['_5_column'].fillna(False, inplace=True)
rotten_tomatoes_reviews['_10_column'].fillna(False, inplace=True)
print(rotten_tomatoes_reviews)

        review_score                                     review_content  \
3              3.5/5  Whether audiences will get behind The Lightnin...   
6                1/4  Harry Potter knockoffs don't come more transpa...   
7              3.5/5  Percy Jackson isn't a great movie, but it's a ...   
8                  B                         Fun, brisk and imaginative   
9                3/5  Crammed with dragons, set-destroying fights an...   
...              ...                                                ...   
1130004          5/5                         The movie is a revelation.   
1130013        3.5/5  Seen today, it's not only a startling indictme...   
1130014           B+  A rousing visual spectacle that's a prequel of...   
1130015        3.5/5  A simple two-act story: Prelude to war, and th...   
1130016            C  Rides the line between being a pure artifact o...   

        numerator  denominator  bool_denominator  _4_column  _5_column  \
3             3.5        

In [9]:
#common denomiator = to 20
def convert_denominator(row):
    adjusted_score = None  # Define a default value
    if row['_4_column'] == True or row['_5_column'] == True or row['_10_column'] == True:
        row['numerator'] = pd.to_numeric(row['numerator'], errors='coerce').astype(int)

    if row['_4_column'] == True:
        score = 5*row['numerator']
        adjusted_score = f"{score}/20"

    elif row['_5_column'] == True: 
        score = 4*row['numerator']
        adjusted_score = f"{score}/20"

    elif row['_10_column'] == True:
        score = 2*row['numerator']
        adjusted_score = f"{score}/20"

    
    return adjusted_score

# Assuming 'bool_denominator' is already defined in your DataFrame
rotten_tomatoes_reviews['review_score_adjusted'] = rotten_tomatoes_reviews.apply(convert_denominator, axis=1)
rotten_tomatoes_reviews = rotten_tomatoes_reviews.drop(rotten_tomatoes_reviews[rotten_tomatoes_reviews['bool_denominator'] == True].index)
print(rotten_tomatoes_reviews.head())

   review_score                                     review_content numerator  \
3         3.5/5  Whether audiences will get behind The Lightnin...       3.5   
6           1/4  Harry Potter knockoffs don't come more transpa...         1   
7         3.5/5  Percy Jackson isn't a great movie, but it's a ...       3.5   
9           3/5  Crammed with dragons, set-destroying fights an...         3   
10          4/5  This action-packed fantasy adventure, based on...         4   

    denominator  bool_denominator  _4_column  _5_column  _10_column  \
3             5             False      False       True       False   
6             4             False       True      False       False   
7             5             False      False       True       False   
9             5             False      False       True       False   
10            5             False      False       True       False   

   review_score_adjusted  
3                  12/20  
6                   5/20  
7          

In [10]:
# Delete unused columns

columns_to_delete = ['numerator','denominator','bool_denominator', '_4_column', '_5_column','_10_column']
rotten_tomatoes_reviews.drop(columns=columns_to_delete, inplace=True)


rotten_tomatoes_reviews.rename(columns={'review_score': 'original_review_score'}, inplace=True)
#rotten_tomatoes_reviews[['numerator', 'denominator']] = rotten_tomatoes_reviews['review_score_adjusted'].str.split('/', expand=True)
#rotten_tomatoes_reviews['numerator'] = rotten_tomatoes_reviews['numerator'].astype(int)



rotten_tomatoes_reviews.to_csv('Rotten_Tomato_Adjusted_Data.csv', index=False)


print(rotten_tomatoes_reviews.head())

   original_review_score                                     review_content  \
3                  3.5/5  Whether audiences will get behind The Lightnin...   
6                    1/4  Harry Potter knockoffs don't come more transpa...   
7                  3.5/5  Percy Jackson isn't a great movie, but it's a ...   
9                    3/5  Crammed with dragons, set-destroying fights an...   
10                   4/5  This action-packed fantasy adventure, based on...   

   review_score_adjusted  
3                  12/20  
6                   5/20  
7                  12/20  
9                  12/20  
10                 16/20  


In [11]:

rotten_tomatoes_reviews_adjusted = pd.read_csv('Rotten_Tomato_Adjusted_Data.csv')
print(len(rotten_tomatoes_reviews_adjusted.index))
rotten_tomatoes_reviews_adjusted.dropna()
print(len(rotten_tomatoes_reviews_adjusted.index))


print(rotten_tomatoes_reviews_adjusted.head())

629257
629257
  original_review_score                                     review_content  \
0                 3.5/5  Whether audiences will get behind The Lightnin...   
1                   1/4  Harry Potter knockoffs don't come more transpa...   
2                 3.5/5  Percy Jackson isn't a great movie, but it's a ...   
3                   3/5  Crammed with dragons, set-destroying fights an...   
4                   4/5  This action-packed fantasy adventure, based on...   

  review_score_adjusted  
0                 12/20  
1                  5/20  
2                 12/20  
3                 12/20  
4                 16/20  


In [12]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mmorr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mmorr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
df = pd.read_csv('Rotten_Tomato_Adjusted_Data.csv', usecols=['review_content'])
lemma = WordNetLemmatizer()
stop_words = stopwords.words('english')
def text_prep(x: str) -> list:
     corp = str(x).lower() 
     corp = re.sub('[^a-zA-Z]+',' ', corp).strip() 
     tokens = word_tokenize(corp)
     words = [t for t in tokens if t not in stop_words]
     lemmatize = [lemma.lemmatize(w) for w in words]
     return lemmatize
preprocess_tag = [text_prep(i) for i in df['review_content']]
df["preprocess_txt"] = preprocess_tag
df['total_len'] = df['preprocess_txt'].map(lambda x: len(x))
file = open('negative_word_list.txt', 'r')
neg_words = file.read().split()
file = open('positive_word_list.txt', 'r')
pos_words = file.read().split()
num_pos = df['preprocess_txt'].map(lambda x: len([i for i in x if i in pos_words]))
df['pos_count'] = num_pos
num_neg = df['preprocess_txt'].map(lambda x: len([i for i in x if i in neg_words]))
df['neg_count'] = num_neg
df['sentiment'] = round((df['pos_count'] - df['neg_count']) / df['total_len'], 2)
df.head()

,review_content,preprocess_txt,total_len,pos_count,neg_count,sentiment
0,Whether audiences will get behind The Lightnin...,"[whether, audience, get, behind, lightning, th...",20,3,2,0.05
1,Harry Potter knockoffs don't come more transpa...,"[harry, potter, knockoff, come, transparent, s...",12,1,0,0.08
2,"Percy Jackson isn't a great movie, but it's a ...","[percy, jackson, great, movie, good, one, trot...",15,3,0,0.20
3,"Crammed with dragons, set-destroying fights an...","[crammed, dragon, set, destroying, fight, thin...",21,1,1,0.00
4,"This action-packed fantasy adventure, based on...","[action, packed, fantasy, adventure, based, ri...",23,3,1,0.09


In [18]:
import pickle
reviews = pd.read_csv('Rotten_Tomato_Adjusted_Data.csv')
reviews[['numerator', 'denominator']] = reviews['review_score_adjusted'].str.split('/', expand=True)

extracted_cl = reviews['numerator']


df2 = pd.concat([df, extracted_cl.rename("review_score")], axis=1)

# Load the saved models
with open('model_log_reg', 'rb') as f:
    log_reg = pickle.load(f)

with open('model_tfidf', 'rb') as f:
    tfidf = pickle.load(f)

# Read data from the file

# Assuming your DataFrame has a column 'review' containing text data
text_data = df2['review_content']

# TF-IDF vectorization
text_data_vectorized = tfidf.transform(text_data)

# Make predictions
predictions = log_reg.predict(text_data_vectorized)

# Create a new column with the predicted sentiments
df2['predicted_sentiment'] = predictions

df2['predicted_sentiment'] = df2['predicted_sentiment'].map({'positive': 1.0, 'negative': 0.0})

# Display the DataFrame with the new column
print(df2.head())
df2.dropna()

df2.to_csv('Rotten_Tomato_Sentiment.csv', index=False)




                                      review_content  \
0  Whether audiences will get behind The Lightnin...   
1  Harry Potter knockoffs don't come more transpa...   
2  Percy Jackson isn't a great movie, but it's a ...   
3  Crammed with dragons, set-destroying fights an...   
4  This action-packed fantasy adventure, based on...   

                                      preprocess_txt  total_len  pos_count  \
0  [whether, audience, get, behind, lightning, th...         20          3   
1  [harry, potter, knockoff, come, transparent, s...         12          1   
2  [percy, jackson, great, movie, good, one, trot...         15          3   
3  [crammed, dragon, set, destroying, fight, thin...         21          1   
4  [action, packed, fantasy, adventure, based, ri...         23          3   

   neg_count  sentiment review_score  predicted_sentiment  
0          2       0.05           12                  1.0  
1          0       0.08            5                  0.0  
2          0  

In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def analyze_sentiment(comment: str) -> pd.DataFrame:
    lemma = WordNetLemmatizer()
    stop_words = stopwords.words('english')

    def text_prep(x: str) -> list:
        corp = str(x).lower() 
        corp = re.sub('[^a-zA-Z]+',' ', corp).strip() 
        tokens = word_tokenize(corp)
        words = [t for t in tokens if t not in stop_words]
        lemmatize = [lemma.lemmatize(w) for w in words]
        return lemmatize

    # Create a DataFrame with the provided comment
    df = pd.DataFrame({'review_content': [comment]})

    # Apply text preprocessing to the comment
    preprocess_tag = [text_prep(i) for i in df['review_content']]
    df["preprocess_txt"] = preprocess_tag
    df['total_len'] = df['preprocess_txt'].map(lambda x: len(x))

    # Load positive and negative word lists
    file = open('negative_word_list.txt', 'r')
    neg_words = file.read().split()
    file = open('positive_word_list.txt', 'r')
    pos_words = file.read().split()

    # Count positive and negative words in the comment
    num_pos = df['preprocess_txt'].map(lambda x: len([i for i in x if i in pos_words]))
    df['pos_count'] = num_pos
    num_neg = df['preprocess_txt'].map(lambda x: len([i for i in x if i in neg_words]))
    df['neg_count'] = num_neg

    # Compute sentiment score
    df['sentiment'] = round((df['pos_count'] - df['neg_count']) / df['total_len'], 2)

    return df

# Load the data
rotten_tomatoes_reviews_adjusted = pd.read_csv('Rotten_Tomato_Sentiment.csv')


# Drop rows with missing values in relevant columns
rotten_tomatoes_reviews_adjusted = rotten_tomatoes_reviews_adjusted.dropna(subset=['sentiment', 'neg_count', 'pos_count', 'total_len', 'review_content'])

# Split the data
train, test = train_test_split(rotten_tomatoes_reviews_adjusted, test_size=0.25, train_size=0.75, random_state=42)

# Fill NaN values in 'review_content' with an empty string
train['review_content'].fillna('', inplace=True)
test['review_content'].fillna('', inplace=True)

# Define features and labels
feature_cols = ['sentiment', 'neg_count', 'pos_count']
train_x, train_y = train[feature_cols], train['review_content']
test_x, test_y = test[feature_cols], test['review_content']

# TF-IDF vectorization for the 'review_content' column
tfidf = TfidfVectorizer(stop_words='english')
train_x_text_vector = tfidf.fit_transform(train['review_content'])
test_x_text_vector = tfidf.transform(test['review_content'])

# Concatenate the text vectorization with the other features
train_x = pd.concat([pd.DataFrame(train_x_text_vector.toarray()), train_x.reset_index(drop=True)], axis=1)
test_x = pd.concat([pd.DataFrame(test_x_text_vector.toarray()), test_x.reset_index(drop=True)], axis=1)

# Define and train the model (using Linear Regression for regression task)
linear_reg = LinearRegression()
linear_reg.fit(train_x, train_y)

# Evaluate the model
predictions = linear_reg.predict(test_x)
mse = mean_squared_error(test_y, predictions)
print(f"Mean Squared Error: {mse}")

# Make predictions for a sample review
sample_review = 'I did not like this movie at all'
sample_sentiment = analyze_sentiment(sample_review)['sentiment'][0]
sample_review_vector = tfidf.transform([sample_review])
sample_review_features = pd.DataFrame([[sample_sentiment, 1, 0, len(sample_review)]], columns=feature_cols)
sample_review_features_vector = pd.concat([pd.DataFrame(sample_review_vector.toarray()), sample_review_features.reset_index(drop=True)], axis=1)
prediction = linear_reg.predict(sample_review_features_vector)
print(f"Predicted Score: {prediction[0]}")


MemoryError: Unable to allocate 322. GiB for an array with shape (466285, 92545) and data type float64

In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import pickle
from math import floor

# Function to analyze sentiment
def analyze_sentiment(comment: str) -> pd.DataFrame:
    lemma = WordNetLemmatizer()
    stop_words = stopwords.words('english')

    def text_prep(x: str) -> list:
        corp = str(x).lower() 
        corp = re.sub('[^a-zA-Z]+',' ', corp).strip() 
        tokens = word_tokenize(corp)
        words = [t for t in tokens if t not in stop_words]
        lemmatize = [lemma.lemmatize(w) for w in words]
        return lemmatize

    # Create a DataFrame with the provided comment
    df = pd.DataFrame({'review_content': [comment]})

    # Apply text preprocessing to the comment
    preprocess_tag = [text_prep(i) for i in df['review_content']]
    df["preprocess_txt"] = preprocess_tag
    df['total_len'] = df['preprocess_txt'].map(lambda x: len(x))

    # Load positive and negative word lists
    file = open('negative_word_list.txt', 'r')
    neg_words = file.read().split()
    file = open('positive_word_list.txt', 'r')
    pos_words = file.read().split()

    # Count positive and negative words in the comment
    num_pos = df['preprocess_txt'].map(lambda x: len([i for i in x if i in pos_words]))
    df['pos_count'] = num_pos
    num_neg = df['preprocess_txt'].map(lambda x: len([i for i in x if i in neg_words]))
    df['neg_count'] = num_neg

    # Compute sentiment score
    df['sentiment'] = round((df['pos_count'] - df['neg_count']) / df['total_len'], 2)

    return df

# Load the data
rotten_tomatoes_reviews_adjusted = pd.read_csv('Rotten_Tomato_Sentiment.csv')

# Drop rows with missing values in relevant columns
rotten_tomatoes_reviews_adjusted = rotten_tomatoes_reviews_adjusted.dropna(subset=['sentiment', 'neg_count', 'pos_count', 'total_len', 'review_content', 'review_score'])

# Ensure 'review_score' is treated as a numerical column
rotten_tomatoes_reviews_adjusted['review_score'] = pd.to_numeric(rotten_tomatoes_reviews_adjusted['review_score'], errors='coerce')
rotten_tomatoes_reviews_adjusted = rotten_tomatoes_reviews_adjusted.dropna(subset=['review_score'])

# Fill NaN values in 'review_content' with an empty string
rotten_tomatoes_reviews_adjusted['review_content'].fillna('', inplace=True)

# Split the data
train, test = train_test_split(rotten_tomatoes_reviews_adjusted, test_size=0.25, train_size=0.75, random_state=42)

# Define features and labels
feature_cols = ['sentiment' 'neg_count', 'pos_count']
train_y = train['review_score']
test_y = test['review_score']

# TF-IDF vectorization for the 'review_content' column
max_features = 500  # Adjust as needed
tfidf = TfidfVectorizer(stop_words='english', max_features=max_features)

# Split the training data into batches (adjust the batch size as needed)
batch_size = 500
num_batches = len(train) // batch_size

# Initialize Linear Regression model
linear_reg = LinearRegression()

previous_mse = float('inf')

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    train_batch = train.iloc[start_idx:end_idx]

    # Vectorize the current batch
    train_batch_text_vector = tfidf.fit_transform(train_batch['review_content'])
    train_batch_x_text = pd.DataFrame(train_batch_text_vector.toarray(), columns=[str(x) for x in range(max_features)])  # Convert to DataFrame with string column names
    train_batch_x_other = train_batch[feature_cols].reset_index(drop=True)
    train_batch_x = pd.concat([train_batch_x_text, train_batch_x_other], axis=1)
    train_batch_y = train_batch['review_score']  # Define train_batch_y

    # Train the model on the current batch
    linear_reg.fit(train_batch_x, train_batch_y)

    # Evaluate the model on the test set
    test_x_text_vector = tfidf.transform(test['review_content'])
    test_x_text = pd.DataFrame(test_x_text_vector.toarray(), columns=[str(x) for x in range(max_features)])  # Convert to DataFrame with string column names
    test_x_other = test[feature_cols].reset_index(drop=True)
    test_x = pd.concat([test_x_text, test_x_other], axis=1)
    predictions = linear_reg.predict(test_x)
    mse = mean_squared_error(test_y, predictions)
    print(f"Mean Squared Error (Batch {i + 1}/{num_batches}): {mse}")
    mse = floor(mse) 
    if mse < 300:
        print(f"Stopping training as Mean Squared Error is less than 300: {mse}")
        break

    # Check if mean squared error is not decreasing significantly
    
filename = 'finalized_model.sav'
pickle.dump(linear_reg, open(filename, 'wb'))



KeyError: "['sentimentneg_count'] not in index"

In [39]:
loaded_model = pickle.load(open(filename, 'rb'))

# Make predictions for a sample review
sample_review = 'I did not like this movie at all'
sample_sentiment = analyze_sentiment(sample_review)['sentiment'][0]
sample_review_vector = tfidf.transform([sample_review])
sample_review_features_text = pd.DataFrame(sample_review_vector.toarray(), columns=[str(x) for x in range(max_features)])

# Define additional features for the sample review
sample_review_features_other = pd.DataFrame([[sample_sentiment, 0, 1, len(sample_review)]], columns=['neg_count', 'pos_count', 'sentiment', 'review_length'])

# Reorder the columns to match the order during training
sample_review_features_other = sample_review_features_other[['neg_count', 'pos_count', 'sentiment', 'review_length']]

# Concatenate the dataframes
sample_review_features = pd.concat([sample_review_features_text, sample_review_features_other], axis=1)

# Make predictions using the loaded model
prediction = loaded_model.predict(sample_review_features)
print(f"Predicted Score: {prediction[0]}")


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- review_length
- sentiment


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

# Function to analyze sentiment
def analyze_sentiment(comment: str) -> pd.DataFrame:
    lemma = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    def text_prep(x: str) -> list:
        corp = str(x).lower()
        corp = re.sub('[^a-zA-Z]+', ' ', corp).strip()
        tokens = word_tokenize(corp)
        words = [t for t in tokens if t not in stop_words]
        lemmatize = [lemma.lemmatize(w) for w in words]
        return lemmatize

    # Create a DataFrame with the provided comment
    df = pd.DataFrame({'review_content': [comment]})

    # Apply text preprocessing to the comment
    preprocess_tag = [text_prep(i) for i in df['review_content']]
    df["preprocess_txt"] = preprocess_tag
    df['total_len'] = df['preprocess_txt'].map(lambda x: len(x))

    # Load positive and negative word lists
    with open('negative_word_list.txt', 'r') as file:
        neg_words = file.read().split()
    with open('positive_word_list.txt', 'r') as file:
        pos_words = file.read().split()

    # Count positive and negative words in the comment
    num_pos = df['preprocess_txt'].map(lambda x: len([i for i in x if i in pos_words]))
    df['pos_count'] = num_pos
    num_neg = df['preprocess_txt'].map(lambda x: len([i for i in x if i in neg_words]))
    df['neg_count'] = num_neg

    # Compute sentiment score
    df['sentiment'] = round((df['pos_count'] - df['neg_count']) / df['total_len'], 2)
    
    return df

# Load the data
rotten_tomatoes_reviews_adjusted = pd.read_csv('Rotten_Tomato_Sentiment.csv')

# Drop rows with missing values in relevant columns
rotten_tomatoes_reviews_adjusted = rotten_tomatoes_reviews_adjusted.dropna(subset=['sentiment', 'neg_count', 'pos_count', 'total_len', 'predicted_sentiment', 'review_score'])

# Split the data
train, test = train_test_split(rotten_tomatoes_reviews_adjusted, test_size=0.25, train_size=0.75, random_state=42)

# Define features and labels
feature_cols = ['sentiment', 'neg_count', 'pos_count', 'total_len', 'predicted_sentiment']
train_y = train['review_score']
test_y = test['review_score']

# Define additional features
train_x_other = train[feature_cols].reset_index(drop=True)
test_x_other = test[feature_cols].reset_index(drop=True)

# Define and train the model (using Linear Regression for regression task)
linear_reg = LinearRegression()
linear_reg.fit(train_x_other, train_y)

# Evaluate the model
predictions = linear_reg.predict(test_x_other)
mse = mean_squared_error(test_y, predictions)
print(f"Mean Squared Error: {mse}")

with open ('model_sentiment_values', 'wb') as files:
        pickle.dump(linear_reg, files)

Mean Squared Error: 15.678193400070688


In [ ]:
# Make predictions for a sample review
with open('model_sentiment_values' , 'rb') as f:
    linear_reg = pickle.load(f)
sample_review = 'I did not like this movie at all'
sample_df = analyze_sentiment(sample_review)[feature_cols]
prediction = linear_reg.predict(sample_df)
print(f"Predicted Score: {prediction[0]}")
